<a href="https://colab.research.google.com/github/SY-256/llms-from-scratch/blob/main/notebooks/ch03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter3 Attentionメカニズムのコーディング

In [ ]:
import torch

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

In [ ]:
# 2つ目のトークンをクエリとして使う
query = inputs[1]

attn_scores_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i, query) # ドット積
print(attn_scores_2)

### ドット積：

ドット積とは、基本的には、2つのベクトルの要素ごとに掛け合わせ、その積の総和を求める簡潔な方法

2つのベクトルを結合してスカラー値を得るための数学的な手段であるだけでなく、ベクトル同士の類似度の尺度でもある。なぜなら、2つのベクトルがどれくらい密に並んでいるか（どのくらい同じ方向を向いているか）を定量化するからである。ベクトル間のドット積が大きいほど、ベクトル間の類似度は高くなる。

ドット積はシーケンスの各要素が他の要素にどれくらい注目しているか（注意を払っている度合い）を決定する。ドット積が大きいほど、2つの要素は類似していると見なされ、それらの間のAttentionスコアも高くなる。

In [ ]:
# ドット積
res = 0
for idx, element in enumerate(inputs[0]):
    res += inputs[0][idx] * query[idx]

# 要素ごとの積の和が、ドット積（torch.dot()）と同じになる
print(res)
print(torch.dot(inputs[0], query))

In [ ]:
# Attentionスコアの正規化 -> 総和が1になるようにする
# 正規化することで、LLMにおいて重みの解釈を容易にし、学習の安定性を維持するのに役立つ慣例となる
attn_weights_2_tmp = attn_scores_2 / attn_scores_2.sum()

print("Attention weights: ", attn_weights_2_tmp)
print("Sum: ", attn_weights_2_tmp.sum())

In [ ]:
# 正規化にはソフトマックス関数を使用するのが一般的
# 極端な値を上手く扱うのに適しており、訓練時の勾配特性をより安定させる
# ソフトマックス関数では、Attentionの重みが常に正になる

def softmax_navive(x):
    return torch.exp(x) / torch.exp(x).sum()

attn_weights_2_navive = softmax_navive(attn_scores_2)
print("Attention weights: ", attn_weights_2_navive)
print("Sum: ", attn_weights_2_navive.sum())

In [ ]:
# torchのsoftmax関数（これ使う）
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)
print("Attention weights: ", attn_weights_2)
print("Sum: ", attn_weights_2.sum())

In [ ]:
# 得られたベクトルをすべて加算する（加重和）
query = inputs[1]

context_vec_2 = torch.zeros(query.shape)
for i, x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i]*x_i # 1x5 * 5x3 = 1x3 の行列が出力
print(context_vec_2)

In [ ]:
# すべての入力のペアについてドット積を計算
# Step1: Attentionスコアを計算
attn_scores = torch.empty(6, 6)

for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i, j] = torch.dot(x_i, x_j)

print(attn_scores)

In [ ]:
# 行列積を使えば簡単に求まるよ
attn_scores = inputs @ inputs.T
print(attn_scores)

In [ ]:
# Step2: SoftMax関数で正規化 -> Attentionの重みを求める
# 入力パラメータdim: 入力テンソルのどの次元に沿って計算を行うか
# dim=-1に設定すると、入力テンソルの最後の次元に沿って正規化するようになる
attn_weights = torch.softmax(attn_scores, dim=-1)
print(attn_weights)

In [ ]:
# 2次元テンソルは列ごとに正規化が行われ、各行（列次元）値の合計が1になる
row_2_sum = sum(attn_weights[1])
print("Row 2 sum: ", row_2_sum)
print("All row sums: ", attn_weights.sum(dim=-1))

In [ ]:
# Step3: Step2のAttentionの重みと入力テンソルの行列積を使って、
# すべてのコンテキストベクトルを計算
all_context_vecs = attn_weights @ inputs
print(all_context_vecs)

In [ ]:
# 前に計算した2行目の値と比較
print("Previous 2nd context vector: ", context_vec_2)

## 3.4 訓練可能な重みを持つSelf-Attentionを実装する
- 訓練可能な重み行列を適用する

In [ ]:
x_2 = inputs[1] # 2つ目の入力要素
d_in = inputs.shape[1] # 入力埋め込みサイズ（d_in=3）
d_out = 2 # 出力埋め込みサイズ（d_out=2）

GPTモデルでは入力と出力の次元数は通常同じ（今回は違うけど）

In [ ]:
# 3つの重み行列を初期化する
torch.manual_seed(123)

W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_key = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

出力を見やすくするためにrequires_grad=Falseにしているが、モデルの訓練に重み行列を使用する場合は、訓練中にそれらの行列を更新するためにrequires_grad=Trueに設定することになる

In [ ]:
# queryベクトル, keyベクトル, valueベクトルを計算
query_2 = x_2 @ W_query
key_2 = x_2 @ W_key
value_2 = x_2 @ W_value
print(query_2)

### 重みパラメータ vs Attentionの重み
重みパラメータはネットワークに接続（結合）を定義するために学習される基本的な係数であり、Attentionの重みはコンテキストに特化した動的な値（ネットワークが入力の異なる部分に注意を払う度合いを決定する）

In [ ]:
# すべての入力要素のキーベクトルと値ベクトルを計算する
# クエリに対するAttentionの重みの計算に必要になるから
keys = inputs @ W_key
values = inputs @ W_value
print("keys.shape:", keys.shape)
print("values.shape:", values.shape)

6つの入力トークンが3次元から2次元の埋め込み空間に上手く射影されている

In [ ]:
# Attentionスコアの計算
# 正規化していないAttentionのスコア
keys_2 = keys[1]
attn_score_22 = query_2.dot(keys_2)
print(attn_score_22)

In [ ]:
# 行列積を使ってすべてのAttentionスコアを計算するように一般化できる
attn_scores_2 = query_2 @ keys.T # このクエリに対するすべてのAttentionスコア
print(attn_scores_2)

In [ ]:
# Attentionの重み
d_k = keys.shape[-1]
attn_weights_2 = torch.softmax(attn_scores_2 / d_k**0.5, dim=-1) # キーの埋め込み次元の平方根で割って正規化
print(attn_weights_2)

In [ ]:
# コンテキストベクトルを計算
context_vec_2 = attn_weights_2 @ values
print(context_vec_2)

In [ ]:
# コンパクトにSelf-Attentionクラス
import torch.nn as nn

class SelfAttention_v1(nn.Module):

    def __init__(self, d_in, d_out):
        super().__init__()
        self.W_query = nn.Parameter(torch.rand(d_in, d_out))
        self.W_key = nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = nn.Parameter(torch.rand(d_in, d_out))

    def forward(self, x):
        keys = x @ self.W_key
        queries = x @ self.W_query
        values = x @ self.W_value

        attn_scores = queries @ keys.T # ωの計算
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1] ** 0.5, dim=-1
        ) # Attentionの重み行列

        context_vec = attn_weights @ values # コンテキストベクトルを含んでいる行列
        return context_vec

torch.nn.Module: PyTorchのモデル層の作成と管理に必要な機能を適用するPyTorchモデルの基本的な構成要素

In [ ]:
# クラスの使用
# 6つの埋め込みベクトルから6つのコンテキストベクトルを含んでいる行列が得られる
torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in, d_out)
print(sa_v1(inputs))

In [ ]:
# PyTorchのLinear層を利用するSelf-Attentionクラス
class SelfAttention_v2(nn.Module):

    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

    def forward(self, x):
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1] ** 0.5, dim=-1
        )
        context_vec = attn_weights @ values
        return context_vec

In [ ]:
torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in, d_out)
print(sa_v2(inputs))

torch.nn.Linearがより洗練された初期化スキームを使っているために、重み行列の初期値がことなるため、SeltAttention_v1とSelfAttention_v2で出力の値が異なる

In [ ]:
# 練習問題3-1
sa_v1.W_key = nn.Parameter(sa_v2.W_key.weight.T)
sa_v1.W_query = nn.Parameter(sa_v2.W_query.weight.T)
sa_v1.W_value = nn.Parameter(sa_v2.W_value.weight.T)

print(sa_v1(inputs))

In [ ]:
sa_v2.W_key.weight

In [ ]:
sa_v1.W_key.H

In [ ]:
nn.Parameter(sa_v2.W_key.weight.T)

## 3.5 Casual Attentionで未来の単語を隠す
- Masked Attention: 特殊なAttention

与えられたトークンを処理して、Attentionスコアを計算する時に、モデルがシーケンスの前と現在の入力だけを考慮するように制限する。

In [ ]:
# step1. Attentionの重みを計算する
torch.manual_seed(789)
queries = sa_v2.W_query(inputs)
keys = sa_v2.W_key(inputs)

attn_scores = queries @ keys.T
attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
print(attn_weights)

In [ ]:
attn_scores.shape[0]

In [ ]:
# step2. マスクを作成する
# 対角線上の要素を0にする
context_length = attn_scores.shape[0]
mask_simple = torch.tril(torch.ones(context_length, context_length))
print(mask_simple)

このマスクをAttentionの重みにかけると対角成分要素を0にできる

In [ ]:
masked_simple = attn_weights * mask_simple
print(masked_simple)

In [ ]:
# step3. 各行のAttentionの重みが合計で1になるように再び正規化
# 各行の各要素を各行の合計で割ると次元を保ったまま正規化できる（割合計算）
rows_sums = masked_simple.sum(dim=-1, keepdim=True)
masked_simple_norm = masked_simple / rows_sums
print(masked_simple_norm)

In [ ]:
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
masked = attn_scores.masked_fill(mask.bool(), -torch.inf)
print(masked)

In [ ]:
# マスクされた結果にsoftmax関数を適用
attn_weights = torch.softmax(masked / keys.shape[-1]**0.5, dim=-1)
print(attn_weights)

In [ ]:
mask

### Attentionの重みにドロップアウトマスクを適用する
- Attentionの重みを計算した後にドロップアウトを適用 <- 通常こっち
- Attentionの重みを値ベクトルに適用した後にドロップアウトを適用

の2パターンがある

In [ ]:
torch.manual_seed(123)
dropout = torch.nn.Dropout(0.5) # ドロップアウト率として50%を選択
example = torch.ones(6, 6)
print(dropout(example))

ドロップアウトでアクティブな（0に設定されなかった）要素の減少を補うために、行列の残りの要素の値は 1/ 0.5 = 2にスケールアップされる。このスケーリングはAttentionの重みの全体のバランスを維持する上で非常に重要な役割を果たす。

- Attentionメカニズムの平均的な影響力が訓練フェーズと推論フェーズの両方で一貫した情報に保たれるから

In [ ]:
# ドロップアウトをAttention行列に適用
torch.manual_seed(123)
print(dropout(attn_weights))

ドロップアウト後の出力はOSによって異なる場合があるので注意する

### コンパクトなCasual Attentionクラスを実装する

In [ ]:
# それぞれ6つのトークンからなる2つの入力：各トークンの埋め込み次元は3
# バッチ入力をシミュレートするために、入力テキストサンプルを複製
batch = torch.stack((inputs, inputs), dim=0)
print(batch.shape)

In [ ]:
# Casual Attentionクラス
class CasualAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, qlv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=False)
        self.W_key = nn.Linear(d_in, d_out, bias=False)
        self.W_value = nn.Linear(d_in, d_out, bias=False)

        self.dropout = nn.Dropout(dropout) # ドロップアウト層を追加
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape # 次元1と2を入れ替える。バッチ次元は最初の位置(0)のまま
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.transpose(1, 2)
        attn_scores.masked_fill_( # PyTorchでは、末尾にアンダースコアを持つ演算はインプレースで実行され、無駄なメモリコピーが回避される
            self.mask.bool()[:num_tokens, :num_tokens], -torch.inf
        )
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )
        attn_weights = self.dropout(attn_weights)

        context_vec = attn_weights @ values
        return context_vec

In [ ]:
# Casual Attentionモデルの利用
torch.manual_seed(123)
context_length = batch.shape[1]
ca = CasualAttention(d_in, d_out, context_length, 0.0)
context_vecs = ca(batch)
print(context_vecs)
print("context_vecs.shape: ", context_vecs.shape)

## 3.6 SIngle-head AttentionをMulti-head Attentionに拡張する
- Casual Attentionクラスを複数のヘッドで拡張
- 複数のCasual Attentionモジュールを積み重ねることで、Multi-head Attentionを直感的に構築
- Multi-head Attentionの主な考え方：学習済みの異なる線形射影でAttentionメカニズムを複数回（並列に）実行するということ
- 学習済み線形射影とは、入力データ（Attentionのメカニズムのクエリベクトル、キーベクトル、値ベクトルなど）に重み行列をかけた結果のこと

In [ ]:
# Multi-head Attentionを実装するラッパークラス
class MultiHeadAttentionWrapper(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        self.heads = nn.ModuleList(
            [CasualAttention(d_in, d_out, context_length, dropout, qkv_bias)
            for _ in range(num_heads)]
        )

    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)

In [ ]:
# MultiHeadAttnetionクラスの利用
torch.manual_seed(123)
context_length = batch.shape[1] # トークン数
d_in, d_out = 3, 2
mha = MultiHeadAttentionWrapper(d_in, d_out, context_length, 0.0, num_heads=2)
context_vecs = mha(batch)

print(context_vecs)
print(f"context_vecs.shape: {context_vecs.shape}")

出力の2つのコンテキストベクトルの値がまったく一緒なのは、入力テキスト(`batch`)を複製しているから

ループで逐次処理されるので、複数のヘッドを並列に処理できれば実装をさらに改善できるはず

In [ ]:
# 練習問題3-2
torch.manual_seed(123)
context_length = batch.shape[1] # トークン数
d_in, d_out = 3, 1
mha = MultiHeadAttentionWrapper(d_in, d_out, context_length, 0.0, num_heads=2)
context_vecs = mha(batch)

print(context_vecs)
print(f"context_vecs.shape: {context_vecs.shape}")

### 重み分割を実装することで効率的にMulti-head Attentionを実装する

In [ ]:
# 効率的なMulti-head Attentionクラスの実装
class MultiHeadAttention(nn.Module):
    def __ini__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0) # 出力に次元数がヘッド数と割り切れない場合は警告出す

        self.d_out = d_out
        self.num_heads = num_heads